## 1

In [1]:
from collections import defaultdict

def mapper(lines):
    mapped = []
    for line in lines:
        for word in line.lower().split():
            mapped.append((word, 1))
    return mapped

def reducer(mapped_data):
    reduced = defaultdict(int)
    for word, count in mapped_data:
        reduced[word] += count
    return reduced

if __name__ == "__main__":
    file_path = r"input.txt"

    with open(file_path, "r", encoding="utf-8") as f:
        data = f.readlines()

    mapped = mapper(data)
    reduced = reducer(mapped)

    print("Final Output (Word Count):\n")
    for k, v in reduced.items():
        print(k, v)


Final Output (Word Count):

hello 1
iam 1
sai 1
vamsi 1


## 2

In [9]:
import pandas as pd

df = pd.read_csv("students.csv")

print("Full Table:")
print(df)

print("\nStudents with marks > 70:")
print(df[df["marks"] > 70])


Full Table:
      name  marks
0     ABHI     50
1      SAI     60
2     ANJU     70
3    VAMSI     33
4   RAJANI     88
5   GANESH     89
6    SANJU     90
7   YAMINI     94
8  VASISTA     78
9      RAM     44

Students with marks > 70:
      name  marks
4   RAJANI     88
5   GANESH     89
6    SANJU     90
7   YAMINI     94
8  VASISTA     78


## 3

In [10]:
!pip install uv

  Using cached uv-0.9.26-py3-none-win_amd64.whl.metadata (12 kB)
Using cached uv-0.9.26-py3-none-win_amd64.whl (23.7 MB)


In [16]:
!pip install pyspark

     ---------------------------------------- 0.0/455.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/455.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/455.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/455.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/455.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/455.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/455.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/455.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/455.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/455.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/455.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/455.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/455.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/4

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\saiva\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\saiva\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\saiva\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\saiva\anaconda3\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\saiva\anaconda3\Lib\http\client.py", line 479, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "C:\Users\saiva\anaconda3\Lib\socket.py", line 720, in readinto
    return self._sock.recv_into(b)
           ^

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, avg, max as mx, min as mn

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, avg, max as mx, min as mn

def create_spark_session():
    spark = SparkSession.builder.appName("BigDataCleaning").getOrCreate()
    return spark

def load_data(spark):
    df = spark.read.csv("data.csv", header=True, inferSchema=True)
    return df

def explore_data(df):
    df.show()
    df.printSchema()
    df.describe().show()

def clean_data(df):
    df = df.withColumn("marks", when(col("marks").isNull(), 0).otherwise(col("marks")))
    avg_age = df.select(avg("age")).first()[0]
    df = df.withColumn("age", when(col("age").isNull(), avg_age).otherwise(col("age")))
    df.show()
    return df

def transform_data(df):
    df = df.withColumn("marks", col("marks") + 5)
    df = df.withColumn(
        "grade",
        when(col("marks") >= 90, "A")
        .when(col("marks") >= 75, "B")
        .otherwise("C")
    )
    df.show()
    return df

def analyze_data(df):
    df.select(
        mx("marks").alias("MaxMarks"),
        mn("marks").alias("MinMarks"),
        avg("marks").alias("AvgMarks")
    ).show()

def save_output(df):
    df.coalesce(1).write.mode("overwrite").csv("output_cleaned", header=True)

def main():
    spark = create_spark_session()
    df = load_data(spark)
    explore_data(df)
    df = clean_data(df)
    df = transform_data(df)
    analyze_data(df)
    save_output(df)
    spark.stop()

if __name__ == "__main__":
    main()


## 4

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator

def create_spark():
    spark = SparkSession.builder.appName("BigMLProject").getOrCreate()
    return spark

def load_data(spark):
    df = spark.read.csv("students.csv", header=True, inferSchema=True)
    return df

def explore_data(df):
    df.show()
    df.describe().show()

def prepare_features(df):
    assembler = VectorAssembler(inputCols=["marks"], outputCol="features")
    df = assembler.transform(df)
    df = df.select("features", "result")
    df.show()
    return df

def train_model(df):
    lr = LogisticRegression(labelCol="result", featuresCol="features")
    model = lr.fit(df)
    return model

def evaluate_model(model, df):
    predictions = model.transform(df)
    predictions.select("features", "result", "prediction", "probability").show()
    evaluator = BinaryClassificationEvaluator(labelCol="result")
    accuracy = evaluator.evaluate(predictions)
    print("Model Accuracy:", accuracy)

def main():
    spark = create_spark()
    df = load_data(spark)
    explore_data(df)
    df = prepare_features(df)
    model = train_model(df)
    evaluate_model(model, df)
    spark.stop()

if __name__ == "__main__":
    main()


## 5

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, max as mx, min as mn

def create_spark_session():
    spark = SparkSession.builder.appName("NoSQL_JSON_Processing").getOrCreate()
    return spark

def load_data(spark):
    df = spark.read.json("students.json")
    return df

def explore_data(df):
    df.show()
    df.printSchema()
    df.describe().show()

def analyze_data(df):
    stats = df.select(
        mx(col("marks")).alias("MaxMarks"),
        mn(col("marks")).alias("MinMarks"),
        avg(col("marks")).alias("AvgMarks")
    )
    stats.show()

    branch_avg = df.groupBy("branch").avg("marks")
    branch_avg.show()

    top_student = df.orderBy(col("marks").desc()).limit(1)
    top_student.show()

    return df, branch_avg

def save_output(df, branch_avg):
    df.coalesce(1).write.mode("overwrite").json("output_students_json")
    branch_avg.coalesce(1).write.mode("overwrite").csv("output_branch_avg", header=True)

def main():
    spark = create_spark_session()
    df = load_data(spark)
    explore_data(df)
    df, branch_avg = analyze_data(df)
    save_output(df, branch_avg)
    spark.stop()

if __name__ == "__main__":
    main()


## 6

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def load_data():
    df = pd.read_csv("sales.csv")
    print(df)
    return df

def analyze_data(df):
    print(df.describe())
    print("Max:", df["sales"].max())
    print("Min:", df["sales"].min())
    print("Avg:", df["sales"].mean())

def line_plot(df):
    plt.figure()
    plt.plot(df["month"], df["sales"], marker="o")
    plt.show()

def bar_plot(df):
    plt.figure()
    sns.barplot(x="month", y="sales", data=df)
    plt.show()

def main():
    df = load_data()
    analyze_data(df)
    line_plot(df)
    bar_plot(df)

if __name__ == "__main__":
    main()
